# 🧭torch.autograd
它是 PyTorch 的 自动微分核心模块。理解了 autograd，基本就能掌握 PyTorch 的“自动梯度”原理。

## 1. 概念

autograd = 自动求导引擎

本质：在 前向计算 时，PyTorch 会 动态构建一张计算图 (DAG)，记录张量之间的运算关系；在 反向传播 时，autograd 会从输出往输入回溯，利用 链式法则 自动计算梯度。

适用于 标量 loss 对 模型参数 的梯度计算，是训练神经网络的关键。

---

## 2. autograd 的核心属性
### 👉(1) requires_grad

- `控制一个张量是否需要梯度。`

- `默认 False，需要手动打开：x = torch.ones(3, requires_grad=True)`

- `只有 浮点 / 复数 张量能设置 requires_grad=True。`

### 👉(2) .grad

- `保存 梯度结果（即 ∂loss/∂x）。`

- `只对 叶子节点（leaf tensors） 有意义：`

- 1. `叶子节点 = 直接由用户创建的、requires_grad=True 的张量。`

- 2. `中间节点的 .grad 默认不会存储（节省内存）。`

### 👉(3) .grad_fn

- `每个 非叶子张量 都有一个 .grad_fn 属性，记录它是由哪种函数（操作）创建的。`

例如：
x = torch.ones(2, requires_grad=True)  \
y = x + 2  \
print(y.grad_fn)   

### 👉(4) 计算图 (Dynamic Computation Graph)

- `动态图：每次前向传播时都会重新构建，不像 TensorFlow 1.x 那样要静态编图。`

- `在 backward() 时，这个图会被释放（除非设置 retain_graph=True）。`

---

## 3. autograd 的核心接口
| 函数                                                                                           | 作用                                 |
| -------------------------------------------------------------------------------------------- | ---------------------------------- |
| `torch.autograd.backward(tensors, grad_tensors, ...)`                                        | 核心反向传播接口（大部分时候通过 `.backward()` 调用） |
| `tensor.backward(grad=None, ...)`                                                            | 常用的简化接口                            |
| `torch.autograd.grad(outputs, inputs, grad_outputs, create_graph=False, retain_graph=False)` | 直接返回梯度，而不是累加到 `.grad`，常用于高阶梯度      |
| `torch.autograd.set_detect_anomaly(True)`                                                    | 开启异常检测，定位梯度爆炸/NaN 的来源              |
| `torch.no_grad()`                                                                            | 上下文管理器，临时禁用梯度计算（推理模式）              |
| `torch.enable_grad()`                                                                        | 与 `no_grad` 对应，重新启用梯度              |
| `torch.inference_mode()`                                                                     | 更高效的 **推理模式**，彻底禁用 autograd 记录     |

---

## 4. 运行机制（例子）

In [ ]:
import torch

x = torch.tensor(2.0, requires_grad=True)
y = x ** 2 + 3 * x

# 反向传播
y.backward()
print(x.grad)   # dy/dx = 2*x + 3 = 7


### 流程：

1、前向：构建图 y = x^2 + 3x，保存操作记录。

2、调用 y.backward()：反向遍历计算图。

3、根据链式法则求导：dy/dx = 2x + 3 = 7

4、结果存到 x.grad。

## 5. 高阶用法
### (1) 高阶梯度

In [ ]:
x = torch.tensor(2.0, requires_grad=True)
y = x**3

# 一阶导数
grad1 = torch.autograd.grad(y, x, create_graph=True)[0]

# 二阶导数
grad2 = torch.autograd.grad(grad1, x)[0]
print(grad1, grad2)   # 12, 6


### (2) 禁用梯度（推理/评估时）

In [ ]:
with torch.no_grad():
    y = model(x)  # 不会追踪梯度


### (3) 推理模式（PyTorch 1.9+）

In [ ]:
with torch.inference_mode():
    y = model(x)  # 比 no_grad 更快更省内存


### (4) 只对部分输入求梯度

In [ ]:
a = torch.tensor(2.0, requires_grad=True)
b = torch.tensor(3.0, requires_grad=True)
c = a * b

grad_a = torch.autograd.grad(c, a)[0]  # ∂c/∂a = b
print(grad_a)  # 3


---
## 6. 常见坑 ⚠️
1.梯度会累加\
每次 backward() 会把结果累加到 .grad。训练循环里要 optimizer.zero_grad()。

2.非标量输出必须传梯度 \
y = model(x)   # y 不是标量 \
y.backward()   # ❌ 报错\
y.backward(torch.ones_like(y))   # ✅


3.in-place 操作可能破坏计算图\
对 requires_grad=True 的张量做原地修改可能导致梯度错误。\
❌ x += 1\
✅ x = x + 1

4.整数 / 布尔不能 requires_grad\
只有浮点 / 复数才行。

5.中间结果默认不保存 .grad\
如果想要中间节点的梯度，要用 torch.autograd.grad()。

---

## 7. 学习速查表
# torch.autograd 速查表
---
## 核心属性
- `requires_grad` : 是否追踪梯度
- `.grad`         : 存放梯度结果（仅叶子张量）
- `.grad_fn`      : 创建该张量的 Function
- 计算图 (DAG)   : 前向时动态构建，反向传播时释放
---
## 常用接口
| 函数 | 功能 |
|------|------|
| `tensor.backward()` | 反向传播（标量输出可省略 grad） |
| `torch.autograd.backward()` | 通用接口（支持多个输出） |
| `torch.autograd.grad()` | 返回梯度，不存到 `.grad` |
| `torch.no_grad()` | 临时禁用梯度（推理用） |
| `torch.inference_mode()` | 更快的推理模式 |
| `torch.autograd.set_detect_anomaly(True)` | 开启异常检测 |

---
## 常见坑 ⚠️
1. `.grad` 会累加 → 训练时要 `zero_grad()`
2. 非标量 `.backward()` 需传 `grad`
3. inplace 操作可能破坏计算图
4. 只有浮点/复数张量能 `requires_grad=True`
5. 中间节点默认不保存 `.grad`（用 `autograd.grad()` 获取）

---

# torch.autograd.backward() 
是 PyTorch 自动微分引擎 autograd 的核心接口之一，很多人只知道 loss.backward()，但其实它背后调用的就是 torch.autograd.backward()。

## 1. 定义

In [ ]:
torch.autograd.backward(
    tensors,
    grad_tensors=None,
    retain_graph=None,
    create_graph=False,
    grad_variables=None,   # 旧版本别名
    inputs=None
)


### 它的作用是：
给定一个或多个张量 tensors（通常是 loss 或输出），计算它们对叶子节点（如模型参数）的梯度，并把结果累加到 .grad 属性里。

等价于在张量上调用 .backward()，但更灵活，可以同时对多个目标做反向传播。

---
    
## 2. 参数详解
| 参数                | 类型                     | 说明                                                              |
| ----------------- | ---------------------- | --------------------------------------------------------------- |
| **tensors**       | Tensor 或 list\[Tensor] | 需要反向传播的“终点”（通常是 loss，或者是你指定的函数输出）                               |
| **grad\_tensors** | Tensor 或 list\[Tensor] | 与 `tensors` 形状相同，作为每个输出的“外部梯度” (∂L/∂y)。若标量 loss，可省略；若非标量，必须指定。  |
| **retain\_graph** | bool                   | 是否保留计算图。默认 `False`，一次反向传播后释放；设为 `True` 可多次 backward（如 RNN 训练时）。 |
| **create\_graph** | bool                   | 是否创建更高阶的计算图（允许对梯度再次求导，即高阶导数）。默认 `False`。                        |
| **inputs**        | list\[Tensor]          | 指定只对哪些 Tensor 求梯度。若为 `None`，默认对所有叶子节点计算。                        |

---
## 3. 工作机制

autograd 会把 前向计算过程中构建的计算图 反向遍历。

从 tensors 出发，把 grad_tensors 作为初始梯度（链式法则的起点），一路传播到需要梯度的叶子节点。

最终结果写入各张量的 .grad 属性（注意是 累加 而不是覆盖）。

---
## 4. 常见用法
### (1) 最基本：标量 loss

In [ ]:
import torch

x = torch.ones(2, 2, requires_grad=True)
y = (x * 3).sum()   # 标量
torch.autograd.backward(y)
print(x.grad)


#### 相当于 

In [ ]:
y.backward()

### (2) 非标量张量

必须指定 grad_tensors，否则 PyTorch 不知道“往下传播什么梯度”。

In [ ]:
x = torch.tensor([[1., 2.], [3., 4.]], requires_grad=True)
y = x * 2           # y.shape = (2,2)，不是标量

grad_outputs = torch.ones_like(y)   # 每个元素权重=1
torch.autograd.backward(y, grad_tensors=grad_outputs)
print(x.grad)


### (3) 多个目标一起反传

In [ ]:
x = torch.tensor([1., 2., 3.], requires_grad=True)
y1 = (x**2).sum()     # ∑ x^2
y2 = (x**3).sum()     # ∑ x^3

torch.autograd.backward([y1, y2], grad_tensors=[torch.tensor(1.), torch.tensor(0.1)])
print(x.grad)

👉 意思是：总梯度 = 1·∂y1/∂x + 0.1·∂y2/∂x

### (4) 高阶导数

In [ ]:
x = torch.tensor(2.0, requires_grad=True)
y = x**3

# 一阶梯度
torch.autograd.backward(y, create_graph=True)
dy_dx = x.grad        # 3*x^2 = 12
print(dy_dx)

# 二阶梯度
dy_dx.backward()
print(x.grad)         # 注意 grad 会累加，所以结果是 12 + 12 = 24

👉要避免累加错误，记得 x.grad.zero_()。

### (5) 限制计算某些 inputs 的梯度

In [ ]:
a = torch.tensor(2.0, requires_grad=True)
b = torch.tensor(3.0, requires_grad=True)
c = a * b

torch.autograd.backward(c, inputs=[a])   # 只对 a 求梯度
print(a.grad)   # 3
print(b.grad)   # None


## 5. 常见坑点 ⚠️

1.非标量必须传 grad_tensors

- `y = model(x)  # y.shape=(batch, num_classes)`
- `y.backward()  # ❌ 报错`

✅ 做法：

- `y.backward(torch.ones_like(y))`

2.梯度会累加：
- `每次调用 backward()，结果会累加到 .grad，不是覆盖。训练循环里要 optimizer.zero_grad()。`

3.retain_graph ：
- `如果你在同一个计算图上多次调用 backward()，必须 retain_graph=True，否则第二次会报错。`

4.整数/布尔张量不能反传：
- `只有浮点/复数张量能 requires_grad=True，否则报错。`

5.in-place 操作可能破坏计算图：
- `在 requires_grad=True 的 Tensor 上做 inplace 修改，可能导致梯度计算错误。`

---
## 6. 总结

- `loss.backward() = torch.autograd.backward(loss)（更常用）。`

- `非标量输出要传 grad_tensors。`

- `多个目标 → backward([y1, y2], grad_tensors=[g1, g2])。`

- `高阶导数 → create_graph=True。`

- `梯度累加 → 记得 zero_grad()。`

- `inputs 参数可以只对部分张量求梯度。`